In [1]:
from osgeo import gdal, gdal_array, osr, ogr
import numpy as np
import rasterio
#from affine import Affine
import pyproj

In [24]:
## land cover data
dataset = gdal.Open('../../../zip_extract/Land_Cover/NYC_2017_LiDAR_LandCover.img')
geotransform = dataset.GetGeoTransform()
band = dataset.GetRasterBand(1)

## catch basin data
cb_data = ogr.Open('../../../catch_basins/DEPCatchbasins/DEPCATCHBASINS.dbf')
layer = cb_data.GetLayer()
cb_ref = layer.GetExtent()

In [25]:
# get offset relative to lc 
lc_origin = (geotransform[0],geotransform[3])
cb_origin = (cb_ref[0],cb_ref[3])
offset_ft = (cb_origin[0]-lc_origin[0], cb_origin[1]-lc_origin[1])
offset_pix = (int(offset_ft[0]*2),int(offset_ft[1]*2))


print('Land cover top left georef:',lc_origin)
print('Catch basin top left georef:', cb_origin)
print(f'CB is shifted by {offset_ft} ft')
print(f'Array top left offset: {offset_pix} pixels (array elements)')

Land cover top left georef: (912286.93, 273618.3)
Catch basin top left georef: (913346.4203000069, 271705.03180000186)
CB is shifted by (1059.4903000068152, -1913.2681999981287) ft
Array top left offset: (2118, -3826) pixels (array elements)


In [3]:
def get_lat_long_arrays(band, x_size, y_size, x_offset = 0, y_offset = 0, proj_from = pyproj.Proj(2263), proj_to = pyproj.Proj(proj = 'latlong')): 
    lc_array = gdal_array.BandReadAsArray(band, x_offset, y_offset, x_size, y_size) 
    cols, rows = np.meshgrid(np.arange(x_size), np.arange(y_size))
    rc_en = lambda r,c : (geotransform[0] + (x_offset + c + 0.5) * geotransform[1], geotransform[3] + (y_offset + r + 0.5) * geotransform[5])
    eastings, northings = np.vectorize(rc_en, otypes = [float, float])(rows, cols)
    transformer = pyproj.Transformer.from_proj(proj_from, proj_to)
    longs, lats = transformer.transform(eastings, northings)
    print(f'Land cover values in sample: {np.unique(lc_array)}')
    print(f'Area of sample: {x_size*y_size/4} sq ft')
    print(f'Total pixels (observations) in sample: {x_size*y_size}')
    return lats, longs

def get_coords_from_lat_long(lat, long, proj_from = pyproj.Proj(2263), proj_to = pyproj.Proj(proj = 'latlong')):
    transformer = pyproj.Transformer.from_proj(proj_to, proj_from)
    eastings, northings = transformer.transform(long, lat)
    x = int((eastings - geotransform[0])/geotransform[1]-0.5)
    y = int((northings - geotransform[3])/geotransform[5]-0.5)
    return x, y

def get_array_lat_long_dim(band, lat, long, length, height):
    x_offset, y_offset = get_coords_from_lat_long(lat, long)
    lc_array = gdal_array.BandReadAsArray(band, x_offset, y_offset, length*2, height*2)
    return lc_array

In [ ]:

gdal_array.BandReadAsArray(band, x_offset, y_offset, x_size, y_size) 